# Clase 23

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2023%20-%20Estructuras%20de%20Datos%20I.ipynb)

# Requisitos Previos

* Matemática Básica
* Matemática Discreta

# Disjoint Set Union

La finalidad principal del **Disjoint Set Union** (a partir de ahora DSU) es, como su nombre traducido lo dice, mantener una colección de conjuntos disjuntos de manera que se puedan realizar las siguientes operaciones:

- $Create(x)$: Crea un nuevo conjunto que consta solamente del elemento $x$.

- $Join(x, y)$: Une los conjuntos de los elementos $x$ y $y$ bajo la premisa de que estos son disjuntos.

- $Get(x)$: Devuelve al representante del conjunto $x$.

Dadas las funcionalidades que necesitamos, principalmente la de $Get(x)$, es evidente que deberemos considerar el tener un representante de cada conjunto disjunto. La implementación del representante debe ser tal que si uno realiza consultas sobre los elementos **sin modificaciones sobre el conjunto** entonces el resultado debe ser el mismo siempre.

Analizaremos las implementaciones y técnicas considerando que se realizarán $n$ operaciones de tipo $Create$ y $m$ operaciones en total. Consideraremos que se realizarán las $n$ operaciones $Create$ al inicio y luego se darán las operaciones de tipo $Join(x,y)$ y $Get(x)$. Notemos que la cantidad de operaciones de tipo $Join(x,y)$ está acotada por $n - 1$, pues luego de cada unión se reduce la cantidad de componentes en $1$.

## Usando listas

Si consideramos que los conjuntos estén representados por listas enlazadas, podemos definir que el representante del conjunto sea el primer elemento de la lista. De esta forma, podemos realizar las operaciones de $Create(x)$ y $Get(x)$ en $O(1)$. A pesar de todo, tendremos que analizar cómo realizar el $Join(x, y)$ de manera eficiente:

### Unión natural

Si unimos el conjunto de $x$ con el de $y$ podemos simplemente agregar los elementos de la lista del conjunto de $y$ al final de la lista de los elementos del conjunto $x$ y esta acción tomaría $O(1)$ operaciones (basta modificar los punteros del final de la lista de $x$ con el inicio de la lista de $y$). Sin embargo, ya que cada elemento deberá tener un puntero hacia el inicio de la lista, todos los elementos de $y$ deberán ser modificados, lo que tomará una cantidad de $O(|y|)$ operaciones.

No es difícil notar que el peor de los casos se da cuando el conjunto $y$ aumenta de $1$ en $1$ su tamaño, de esta forma obtendríamos una cantidad de operaciones por cada iteración:

$$ Join(1, 2) \rightarrow 1 $$
$$ Join(3, 2) \rightarrow 2 $$
$$ \vdots $$
$$ Join(n, n-1) \rightarrow n-1 $$

Y notamos que este procedimiento tomaría una complejidad de $O(n^{2} + m)$, lo cual no es muy eficiente.

### Heurística de unión por pesos

Plantearemos una mejora para el Join natural, considerando que las listas llevan también su longitud (lo cual es fácil de mantener y no aumenta la complejidad): Cada vez que unamos dos listas, la de menor longitud será agregada a la de mayor tamaño; en caso de empate, agregaremos una a la otra arbitrariamente. A esta mejora la llamaremos **heurística de unión por pesos**

Probaremos que si consideramos la mejora anterior podremos reducir la complejidad a $O(m + n\log{n})$.

**Teorema:** Al usar la representación por listas de los conjuntos disjuntos y la heurística de unión por pesos, una secuencia de $m$ operaciones de los tipos $Create$, $Join$ y $Get$, de las cuales $n$ son de tipo $Create$, tiene complejidad de $O(m + n\log{n})$.

**Prueba:**

Ya que las operaciones de $Create$ y $Get$ toman $O(1)$, analizaremos el tiempo total que demoraría hacer lsa $n-1$ operaciones $Join$:

Consideremos la cantidad de veces que un solo elemento $a$ actualiza su padre (ya que en este caso, este elemento sería parte de la lista de menor tamaño), sea el mínimo tamaño posible del conjunto resultante luego de la $i$-ésima actualización de $a$ igual a la función $T(i)$, entonces:

$$ T(1) = 2 $$

Ya que en la primera modificación basta con unir un conjunto unitario con el de $a$. Además, debido a que en caso de empates los conjuntos se unen de manera arbitraria, basta tener un conjunto de igual tamaño que el de $a$ para que este elemento sea modificado, así que para la $i$-ésima modificación, el tamaño del conjunto de $a$ tendrá al menos $T(i-1)$ elementos, mientras que el otro conjunto con el cual se lograría dicha modificación también tendrá al menos $T(i-1)$ elementos. Entonces, como deseamos el mínimo, tendremos que:

$$ T(i) = 2T(i-1) $$

No es difícil notar que $T(i) = 2^{i}$, de forma que se necesitarán $2^{i}$ elementos como mínimo para poder modificar $i$ veces un elemento $a$, esto nos permite notar que el limite superior para dicha cantidad de modificaciones es $O(\log{n})$.

De esta manera hemos probado que para todos los elementos se realizará un trabajo total de $O(\log{n})$ por cada uno, así que el aporte de todas las operaciones $Join$ será $O(n\log{n})$.

Finalmente, considerando los demás aportes de las operaciones, tendremos una complejidad de $O(m + n\log{n})$.

## Disjoint set forests

Para optimizar el rendimiento, consideraremos que cada conjunto tendrá una estructura de árbol, en la que todos los elementos tienen asignado un nodo padre almacenado en el arreglo $par$, de manera que el único que no tiene un padre definido es el representante del conjunto $r$ y además para cada elemento $x$ se cumple que existe un camino desde $x$ hasta $r$ yéndose cada vez hacia el padre del nodo.

![Imagen](https://algocoding.files.wordpress.com/2014/09/uf1_intro2.png)

La imagen anterior muestra un ejemplo de la colección de conjuntos:

$$ S = \{\{1, 2, 3\}, \{4, 5\}, \{6\},\{7\}\} $$

Y los representantes de los conjuntos son $\{2, 5, 6, 7\}$ respectivamente.

Con esta implementación, la función $Create$ crea un solo nodo con padre nulo o apuntándose a si mismo como representante, la función $Get$ llega al representante mediante el camino de padres y la función $Join$ hace que el representante de alguno de los conjuntos apunte al representante del otro. De esta manera obtenemos un cambio ligero en las complejidades de $O(1)$ en $Create$ y $Join$, pero un $O(n)$ en la función $Get$, ya que se pueden dar las uniones de forma que el árbol final sea un camino.

En este caso, no hay muchas diferencias respecto a la complejidad que se da al usar listas excepto por la función que debemos optimizar. Notemos que la función $Get$ no tiene un límite de veces a ser ejecutada, pero si podemos optimizar esta función, nuestro trabajo estará terminado.

### Heurísitica de unión por rango

De manera similar a la heurística de unión por peso usada en la implementación con listas, podemos considerar llevar en los representantes del conjunto una variable $sizes$ que va a representar algo similar a un "peso" para los conjuntos (lo más usual es llevar la altura del árbol o la cantidad de elementos, ambos funcionan de manera similar pero el análisis se dará considerando la altura) y de esta manera agregaremos el conjunto de menor peso al de mayor peso.

Análogamente, la complejidad que se obtiene al usar esta heurística de manera única es de $O(\log{n})$ por cada operación del tipo $Get$, ya que la altura del árbol está acotada por $O(\log{n})$ y por ende también la longitud de los caminos hasta el representante.

### Heurística de compresión de caminos

Esta heurística es bastante simple: Cada vez que calculemos el representante de clase para un cierto elemento, asignaremos su valor como nuevo padre. En términos de lo que sucede en el árbol, cada vez que obtengamos el representante, todos los nodos del camino que hemos recorrido para llegar a él serán ahora sus hijos directos para que, en una siguiente llamada, la cantidad de pasos a realizar sea reducida a $1$ si el representante no varía.

El usar esta heurística de manera única nos permite obtener una complejidad de $O(\log{n})$ para cada operación $Get$, Tarjan y Leeuwen analizaron diferentes formas de diseño de rangos y de compresión de caminos en su publicación [Worst-case Analysis of Set Union Algorithms](https://sci-hub.tw/https://doi.org/10.1145/62.2160)

Se puede probar que la complejidad al usar **ambas** heurísticas es de $O(m\alpha(n))$, donde $\alpha(n)$ es la inversa de la función de Ackermann. En términos prácticos para $n \leq 10^{600}$ se da que $\alpha(n) \leq 4$, así que podemos considerar que la ejecución de las operaciones tendrá un comportamiento casi lineal.

Los algoritmos se verán de la siguiente forma:

```C++
void create(int x){
    par[x] = x;
    sizes[x] = 1;
}

int get(int x){
    return par[x] == x? x : par[x] = get(par[x]);
}

void join(int x, int y){
    x = get(x);
    y = get(y);
    if(x == y) return;
    if(sizes[x] > sizes[y]) swap(x, y);
    par[x] = y;
    sizes[y] += sizes[x];
}
```

## Problemas para practicar

- [Anansi's Cobweb - Reverse](https://acm.timus.ru/problem.aspx?space=1&num=1671)

- [Dish Owner - Implementation](https://www.codechef.com/problems/DISHOWN)

- [Lexicographically minimal string - Implementation](https://www.hackerearth.com/practice/data-structures/disjoint-data-strutures/basics-of-disjoint-data-structures/practice-problems/algorithm/lexicographically-minimal-string-6edc1406/description/)

- [Unbelievable Array - Equivalence](https://toph.co/p/unbelievable-array)

- [Parity tree - Math](https://www.codechef.com/problems/PARITREE)

- [Fill The Matrix - Math](https://www.codechef.com/problems/FILLMTR)

- [Mike and Feet - Ranges](https://codeforces.com/problemset/problem/547/B)

- [Sebi and the corrupt goverment - Counting](https://www.codechef.com/problems/SETELE)

- [Quantity of Strings - Counting](https://codeforces.com/contest/151/problem/D)

- [Magical Strings - Counting](https://www.codechef.com/problems/MAGICSTR)


# Sparse Table 

La estructura llamada **Sparse Table** es una estructura de datos inmutable, es decir, que una vez construida ya no se puede modificar sin tener que reconstruirla desde 0. Se basa en obtener respuestas parciales sobre cantidades de elementos que sean potencias de $2$, ya que de esta forma se podrá obtener la respuesta de un conjunto de $2^{l}$ a partir de las respuestas de los conjuntos de $2^{l-1}$ con $l > 0$.

La justificación para esta idea es que todo número $x$ tiene una representación binaria de no más de $\lceil \log_{2}{x} \rceil$ dígitos, por lo que una consulta en un determinado rango puede ser resuelta considerando solo $\lceil \log_{2}{x} \rceil$ conjuntos diferentes.

En términos de memoria, se suele tomar todos los rangos posibles para cada elemento $i$, de forma que tendremos una tabla de tamaño $n \times \lceil\log_{2}{n}\rceil$ para cada respuesta parcial posible. Es evidente que el preprocesamiento para obtener los resultados tendrá una complejidad de $O(n\log{n}\cdot merge)$, donde $merge$ es la complejidad del cálculo de la respuesta parcial de un orden superior.

Pasos para diseñar una sparse table:

1) Dada una función asociativa $f$, definir el atributo con el que estará relacionado el tamaño de un conjunto para las respuestas parciales

2) Definir la recursión que mediante la cual se procesarán las respuestas parciales

Si $f$ es idempotente respecto a la existencia de los elementos, entonces las consultas en rango se pueden responder en $O(merge)$; si no, en $O(\log{n}\cdot merge)$.

Por otro lado, para responder a las consultas tenemos dos escenarios posibles:

1) $f$ es idempotente: En tal caso, si queremos consultar el rango $[L, R]$, podemos tomar la potencia de 2 más grande tal que $2^{k} \leq R - L + 1$ (esto se logra con `k = 31 - __builtin_clz(R - L + 1)`) y notaremos que la respuesta total se puede calcular uniendo los rangos $[L, L + 2^{k} - 1]$ y $[R - 2^{k} + 1, R]$ que ya estarán almacenados en $ST[L][k]$ y $ST[R-2^{k}+1][k]$, finalmente la complejidad será de $O(merge)$.

2) $f$ **no** es idempotente: En tal caso, podemos iterar desde la mayor a la menor potencia de la representación binaria de $R - L + 1$, de forma que calcularemos la respuesta usando las potencias (que ya estarán calculadas en la tabla) uniéndolas con la actual hasta que ya no tengamos potencias por procesar, esto tomará $O(\log{n})$ pasos y por ende la complejidad total será de $O(\log{n}\cdot merge)$.

## Sparse Table para Range Min/Max Query (idempotente)

En este caso, nuestra función será $\min$ o $\max$, las cuales son idempotentes. El problema principal constará de consultar el mínimo o máximo de los elementos de un arreglo $a$ en el rango de índices $[L,R]$. Notemos que nuestra tabla $ST$ estará definida de la siguiente forma:

$$ ST[i][j] = \text{Minimo de los elementos en el rango }[i, i + 2^{j} - 1] $$

Por lo que podemos plantear nuestra recursión de la siguiente forma:

$$ ST[i][0] = a_{i} $$

$$ ST[i][j] = \min{\{a_{i}, a_{i+1}, \ldots, a_{i + 2^{j} - 1}\}} = \min{\{a_{i}, a_{i+1}, \ldots, a_{i + 2^{j-1} - 1}, a_{i + 2^{j - 1}}, \ldots, a_{i + 2^{j}-1}\}} $$

$$ ST[i][j] = \min{\{\min{\{a_{i}, a_{i+1}, \ldots, a_{i + 2^{j-1} - 1}\}}, \min{\{a_{i + 2^{j - 1}}, \ldots, a_{i + 2^{j}-1}\}\}}} = \min{\{ST[i][j-1], ST[i + 2^{j-1}][j-1]\}} $$

Podemos rellenar la tabla de manera iterativa:

```C++
void STBuild(){
    for(int i = 1; i <= n; i++){
        ST[i][0] = a[i];
    }
    for(int j = 1; 1<<j <= n; j++){
        int dis = 1<<(j-1); // 2^(j-1)
        for(int i = 1; i + 2 * dis - 1 <= n; i++){
            ST[i][j] = min(ST[i][j-1], ST[i+dis][j-1]);
        }
    }
}
```

La consulta, al ser una función idempotente, se puede realizar en $O(1)$ usando el método descrito anteriormente:

```C++
int STQuery(int L, int R){
    int d = R - L + 1;
    int k = 31 - __builtin_clz(d);
    int dis = 1<<k;
    return min(ST[L][k], ST[R-dis+1][k])
}
```

## Sparse Table para Sumas (no idempotente)

En este caso, nuestra función será la adición, la cual **no** es idempotente. El problema principal constará de consultar la suma de los elementos de un arreglo $a$ en el rango de índices $[L,R]$. Notemos que nuestra tabla $ST$ estará definida de la siguiente forma:

$$ ST[i][j] = \text{Suma de los elementos en el rango }[i, i + 2^{j} - 1] $$

Por lo que podemos plantear nuestra recursión de la siguiente forma:

$$ ST[i][0] = a_{i} $$

$$ ST[i][j] = a_{i} + a_{i+1} + \ldots + a_{i + 2^{j} - 1} = a_{i} + a_{i+1} + \ldots + a_{i + 2^{j-1} - 1} + a_{i + 2^{j - 1}} + \ldots, a_{i + 2^{j}-1} $$

$$ ST[i][j] = (a_{i} + a_{i+1} + \ldots + a_{i + 2^{j-1} - 1}) + (a_{i + 2^{j - 1}} + \ldots + a_{i + 2^{j}-1}) = ST[i][j-1] + ST[i + 2^{j-1}][j-1] $$

Podemos rellenar la tabla de manera iterativa:

```C++
void STBuild(){
    for(int i = 1; i <= n; i++){
        ST[i][0] = a[i];
    }
    for(int j = 1; 1<<j <= n; j++){
        int dis = 1<<(j-1); // 2^(j-1)
        for(int i = 1; i + 2 * dis - 1 <= n; i++){
            ST[i][j] = ST[i][j-1] + ST[i+dis][j-1];
        }
    }
}
```

La consulta se puede obtener en $O(\log{n})$ con el método descrito anteriormente:

```C++
int STQuery(int L, int R){
    int d = R - L + 1;
    int ans = 0;
    while(d > 0){
        int k = 31 - __builtin_clz(d);
        ans += ST[L][k];
        L += (1<<k); // Hacemos el salto
        d -= (1<<k); // Le quitamos el maximo bit a la distancia
    }
    return ans;
}
```

## Sparse Table sobre árboles (Binary Lifting)

Un problema conocido en árboles con raíz es hallar el *Lowest Common Ancestor (LCA)* de dos nodos $u$ y $v$, el cual se define como el nodo que es ancestro de $u$ y $v$ que está más alejado de la raíz.

Este problema se puede resolver en $O(\log{n})$ por consulta usando Sparse Table para optimizar el tiempo que nos demoraría subir una cantidad de aristas en el árbol.

Primero, consideremos una Sparse Table definida de la siguiente manera:

$$ ST[i][j] = \text{Ancestro de }i\text{ que está a una distancia de }2^{j} $$

Notemos que solo es necesario calcular el $ST[i][j]$ con $2^{j} \leq level[i]$, donde $level[u]$ es la distancia desde $u$ hasta la raíz y esto podemos procesarlo con un DFS desde la raíz, ya que al llegar a cada nodo tendremos tanto el nivel como su padre (ancestro a distancia $2^{0}$).

Plantearemos la forma de hallar el LCA de dos nodos $u$ y $v$ que estén en el mismo nivel:

1) Si $u = v$ el LCA es $u$.

2) Si $l$ es el LCA de $u$ y $v$, sabemos que está a una distancia $d > 0$ y además que si tomamos los ancestros de $u$ y $v$ a una distancia mayor que $d$ estos valores serán los mismos siempre. Notemos que si definimos un predicado respecto a la distancia $k$ del ancestro tal que sea *Verdadero* cuando $ancestro(u, k) \not = ancestro(v, k)$ y *Falso* cuando $ancestro(u, k) = ancestro(v, k)$, podremos hallar el último $k$ con predicado verdadero y su padre será el LCA de $u$ y $v$.

### Obtener el LCA en $O(\log{n})$

Consideremos el predicado anterior y supongamos que el último verdadero es logrado a una distancia $k$, entonces debemos notar que si $2^{p}$ es la máxima potencia de $2$ tal que $2^{p} \leq k$ (claramente este valor es el bit más significativo de $k$) entonces el predicado de $2^{p+1}$ será falso (sino sería una contradicción respecto al valor de $k$).

Eso significa que si iteramos desde el máximo bit posible y encontramos el primero que da *Verdadero* en su predicado, entonces este bit estará prendido en $k$.

Podemos plantear una inducción rápida para demostrar que siguiendo el método anterior (y a medida que encontramos los bits prendidos también reemplazamos el $u$ y $v$ por sus respectivos ancestros) obtendremos el último *Verdadero* de nuestro predicado, por lo que la respuesta en tal caso será el padre de dichos ancestros. De esta forma, al iterar sobre los bits, tendríamos una complejidad de $O(\log{n})$.

Es sencillo notar que para el caso en el que $u$ y $v$ no estén originalmente en el mismo nivel nos basta con emparejarlos primero y aplicar el algoritmo propuesto anteriormente. Ya que tenemos procesada la tabla de ancestros, podemos emparejarlos en $O(\log{n})$.

Construcción del Sparse Table en $O(n\log{n})$:

```C++
void computeST(int u, int p){
    ST[u][0] = p;
    for(int d = 1; 1<<d <= level[u]; d++){
            ST[u][d] = ST[ST[u][d-1]][d-1]; // Distancia 2^d = Subir 2^(d-1) pasos y luego 2^(d-1) pasos más
    }
}

void DFS(int u, int p = -1){
    computeST(u, p);
    for(int v : G[u]){
        if(v == p) continue;
        level[v] = level[u] + 1;
        DFS(v, u);
    }
}
```

Función que nos devuelve el ancestro a una distancia $d$ de un nodo $u$:

```C++
int goUp(int u, int d){
    while(d > 0){
        int k = 31 - __builtin_clz(d);
        u = ST[u][k];
        d -= (1<<k);
    }
    return u;
}
```

Función para calcular el LCA de dos nodos $u$ y $v$:

```C++
int lca(int u, int v){ // Asumiremos que level[u] >= level[v] y si no, los swapeamos
    if(level[u] < level[v]) swap(u, v);
    u = goUp(u, level[u] - level[v]);
    if(u == v) return u; // Primer caso de igualdad de niveles
    for(int i = 31 - __builtin_clz(n); i >= 0; i--){
        if(ST[u][i] != ST[v][i]){ // Este desplazamiento es Verdadero
            u = ST[u][i];
            v = ST[v][i];
        }
    }
    return ST[u][0]; // El LCA es el padre del último verdadero
}
```


## Problemas para practicar

- [Range Minimum Query](https://www.spoj.com/problems/RMQSQ/)

- [Catapult that ball](https://www.spoj.com/problems/THRBL/)

- [Sereja and D](https://www.codechef.com/problems/SEAD)

- [CGCDSSQ](https://codeforces.com/contest/475/problem/D)

- [R2D2 and Droid Army](https://codeforces.com/contest/514/problem/D)

- [Miraculous](https://www.spoj.com/problems/TNVFC1M/)

- [Turn Off The TV](https://codeforces.com/contest/863/problem/E)

- [Fools and Roads](https://codeforces.com/contest/609/problem/E)